### Import the libraries and load the data

In [1]:
import pandas as pd

# Load the dataframes
df1 = pd.read_csv('csv/tuber_states_022023_df.csv')
df2 = pd.read_csv('csv/tuber_states_032023_df.csv')
df3 = pd.read_csv('csv/tuber_states_012023_df.csv')

# Check if they have the same columns
columns_match = (set(df1.columns) == set(df2.columns)) and (set(df2.columns) == set(df3.columns))

# Output the result and the columns of each dataframe if needed
columns_match

True

### Data Info

In [2]:
# Add a month column to each DataFrame
df1['Month'] = 'February'
df2['Month'] = 'March'
df3['Month'] = 'January'

# Concatenate the DataFrames
combined_df = pd.concat([df1, df2, df3], ignore_index=True)

# Display the combined DataFrame shape and the first few rows to verify
combined_df_shape = combined_df.shape
combined_df_head = combined_df.head()

combined_df_shape

(299, 30)

In [3]:
combined_df.to_csv('csv/tuberculose_010203.csv')

In [4]:
combined_df.head()

,Code,Description,Acre,Alagoas,Amapá,Amazonas,Bahia,Ceará,Distrito Federal,Espírito Santo,Goiás,Maranhão,Mato Grosso,Mato Grosso do Sul,Minas Gerais,Paraná,Paraíba,Pará,Pernambuco,Piauí,Rio Grande do Norte,Rio Grande do Sul,Rio de Janeiro,Rondônia,Roraima,Santa Catarina,Sergipe,São Paulo,Tocantins,Month
0,201010402,BIOPSIA DE PLEURA (POR ASPIRACAO/AGULHA / PLEU...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,February
1,202010120,DOSAGEM DE ACIDO URICO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,February
2,202010201,DOSAGEM DE BILIRRUBINA TOTAL E FRACOES,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,February
3,202010228,DOSAGEM DE CALCIO IONIZAVEL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,February
4,202010260,DOSAGEM DE CLORETO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,February


In [5]:
# Load the dataframes
df4 = pd.read_csv('csv/tuberculose_022023_cent.csv')
df5 = pd.read_csv('csv/tuberculose_032023_cent.csv')
df6 = pd.read_csv('csv/tuberculose_012023_cent.csv')

# Check if they have the same columns
columns_match = (set(df4.columns) == set(df5.columns)) and (set(df4.columns) == set(df6.columns))

# Output the result and the columns of each dataframe if needed
columns_match

True

In [6]:
# Add a month column to each DataFrame
df4['Month'] = 'February'
df5['Month'] = 'March'
df6['Month'] = 'January'

# Concatenate the DataFrames
combined_df_cent = pd.concat([df1, df2, df3], ignore_index=True)

# Display the combined DataFrame shape and the first few rows to verify
combined_df_shape_cent = combined_df_cent.shape
combined_df_head_cent = combined_df_cent.head()

combined_df_shape

(299, 30)

In [7]:
# Drop the unnecessary columns and pivot the table to get the desired structure
# Assuming the user wants to aggregate the data by state and month

# First, let's melt the dataframe to long format
data_long = pd.melt(combined_df_cent, id_vars=["Description", "Month"], var_name="State", value_name="Cases")

# Then, filter out the 'Unnamed: 0', 'Code', and 'Description' columns from the State column as they are not states
data_long_filtered = data_long[~data_long['State'].isin(['Code', 'Description'])]

# Now, pivot the table to get the structure with states as rows and months as columns
data_pivoted = data_long_filtered.pivot_table(index='State', columns='Month', values='Cases', aggfunc='sum').reset_index()

# Sort the months in the correct order
data_pivoted = data_pivoted[['State', 'January', 'February', 'March']]

data_pivoted.head()

Month,State,January,February,March
0,Acre,0.0,0.0,0.0
1,Alagoas,5.0,2.0,4.0
2,Amapá,0.0,10.0,15.0
3,Amazonas,9.0,10.0,9.0
4,Bahia,23.0,121.0,48.0


In [8]:
data_pivoted.to_csv('csv/tuberculose_010203_cent.csv')

### Create the dataframe for region instead of states

In [9]:
# Define a dictionary to map each state to its respective region
state_to_region = {
    'Acre': 'North', 'Alagoas': 'Northeast', 'Amapá': 'North', 'Amazonas': 'North',
    'Bahia': 'Northeast', 'Ceará': 'Northeast', 'Distrito Federal': 'Central-West',
    'Espírito Santo': 'Southeast', 'Goiás': 'Central-West', 'Maranhão': 'Northeast',
    'Mato Grosso': 'Central-West', 'Mato Grosso do Sul': 'Central-West', 'Minas Gerais': 'Southeast',
    'Pará': 'North', 'Paraíba': 'Northeast', 'Paraná': 'South', 'Pernambuco': 'Northeast',
    'Piauí': 'Northeast', 'Rio Grande do Norte': 'Northeast', 'Rio Grande do Sul': 'South',
    'Rio de Janeiro': 'Southeast', 'Rondônia': 'North', 'Roraima': 'North', 'Santa Catarina': 'South',
    'Sergipe': 'Northeast', 'São Paulo': 'Southeast', 'Tocantins': 'North'
}

# Create a new DataFrame to hold the aggregated data by region
data_by_region = pd.DataFrame()

# Include the 'Code', 'Description', and 'Month' columns as they are
data_by_region[['Code', 'Description', 'Month']] = combined_df[['Code', 'Description', 'Month']]

# Aggregate the data by region
for state, region in state_to_region.items():
    if region not in data_by_region.columns:
        # Initialize the region column with zeros if it doesn't exist yet
        data_by_region[region] = 0
    # Add the state's data to its corresponding region
    data_by_region[region] += combined_df[state]

# Display the first few rows of the new dataframe to verify the transformation
data_by_region.head()

,Code,Description,Month,North,Northeast,Central-West,Southeast,South
0,201010402,BIOPSIA DE PLEURA (POR ASPIRACAO/AGULHA / PLEU...,February,0.0,0.0,0.0,4.0,0.0
1,202010120,DOSAGEM DE ACIDO URICO,February,0.0,0.0,0.0,1.0,0.0
2,202010201,DOSAGEM DE BILIRRUBINA TOTAL E FRACOES,February,0.0,0.0,0.0,4.0,0.0
3,202010228,DOSAGEM DE CALCIO IONIZAVEL,February,0.0,0.0,0.0,1.0,0.0
4,202010260,DOSAGEM DE CLORETO,February,0.0,0.0,0.0,1.0,0.0


In [10]:
# Define the path for the new CSV file
output_file_path = 'csv/tuberculose_by_region.csv'

# Export the transformed DataFrame to a new CSV file
data_by_region.to_csv(output_file_path, index=False)

# Return the path to the user
output_file_path

'csv/tuberculose_by_region.csv'

### Total Frequency per 100 thousand habitants

In [11]:
df_region_pop = pd.read_csv('csv/regioes_2022.csv')
# Convert the 'População' column to integers after removing spaces
df_region_pop['População'] = df_region_pop['População'].str.replace(' ', '').astype(int)

# Divide the 'População' column by 100,000
df_region_pop['População_per_100k'] = df_region_pop['População'] / 100000

# Display the updated DataFrame to verify the changes
df_region_pop

,Região,População,População_per_100k
0,Região Sudeste,84847187,848.47187
1,Região Nordeste,54644582,546.44582
2,Região Sul,29933315,299.33315
3,Região Norte,17349619,173.49619
4,Região Centro-Oeste,16287809,162.87809


In [12]:
# Filter the dataframe to only include the relevant columns
df_filtered = data_by_region[['Month', 'North', 'Northeast', 'Central-West', 'Southeast', 'South']]

# Aggregate data by month (assuming the aggregation needed is sum, update based on requirement)
# Since the instruction is not clear on what kind of transformation (sum, average, etc.) is needed for the month's data,
# I will proceed by preparing a pivot table that shows sums for each region per month as an initial approach.

# Pivot the table to have months as columns and regions as rows
pivot_df = df_filtered.melt(id_vars=['Month'], var_name='Region', value_name='Cases').groupby(['Region', 'Month']).sum().unstack(fill_value=0)

# Reset index to have a clean DataFrame format and prepare for the desired format
pivot_df.columns = pivot_df.columns.droplevel(0)  # Drop the top level ('Cases')
pivot_df.reset_index(inplace=True)

# Rearrange columns based on the month order if necessary
months_order = ['January', 'February', 'March']
pivot_df = pivot_df[['Region'] + months_order]

pivot_df

Month,Region,January,February,March
0,Central-West,52.0,45.0,45.0
1,North,110.0,153.0,106.0
2,Northeast,942.0,240.0,135.0
3,South,652.0,724.0,1323.0
4,Southeast,365.0,1190.0,3002.0


In [13]:
# Translation dictionary
translation_dict = {
    "Região Sudeste": "Southeast",
    "Região Nordeste": "Northeast",
    "Região Sul": "South",
    "Região Norte": "North",
    "Região Centro-Oeste": "Central-West",
}

# Apply the translation
df_region_pop['Região'] = df_region_pop['Região'].map(translation_dict)

df_region_pop

,Região,População,População_per_100k
0,Southeast,84847187,848.47187
1,Northeast,54644582,546.44582
2,South,29933315,299.33315
3,North,17349619,173.49619
4,Central-West,16287809,162.87809


In [14]:
# Translate "Região" to "Region" to ensure matching column names for the merge
df_region_pop.rename(columns={"Região": "Region"}, inplace=True)

# Rename "Região" to "Region" to ensure matching column names for the merge, if not already done
df_region_pop.rename(columns={"Região": "Region"}, inplace=True)

# Merge the DataFrames on the "Region" column
merged_df = pd.merge(pivot_df, df_region_pop, on="Region")

# Calculate "% per 100k inhabitants" for January, February, and March, rounding to 2 decimal places
merged_df["January per 100k"] = ((merged_df["January"] / merged_df["População_per_100k"]) * 100).round(2)
merged_df["February per 100k"] = ((merged_df["February"] / merged_df["População_per_100k"]) * 100).round(2)
merged_df["March per 100k"] = ((merged_df["March"] / merged_df["População_per_100k"]) * 100).round(2)

# Select the columns for the final DataFrame
final_df = merged_df[["Region", "January per 100k", "February per 100k", "March per 100k"]]

final_df

,Region,January per 100k,February per 100k,March per 100k
0,Central-West,31.93,27.63,27.63
1,North,63.40,88.19,61.10
2,Northeast,172.39,43.92,24.71
3,South,217.82,241.87,441.98
4,Southeast,43.02,140.25,353.81


In [15]:
final_df.to_csv('csv/tuberculose_region_010203_cent.csv')

In [16]:
# Sum the values of January, February, and March, and assign to 'Total Procedures'
pivot_df['Total Procedures'] = pivot_df[['January', 'February', 'March']].sum(axis=1)

region_data = pivot_df.drop(columns=["January", "February", "March"])

# Reset the index without keeping it
region_data_final = region_data.reset_index(drop=True, inplace=False)

region_data_final

Month,Region,Total Procedures
0,Central-West,142.0
1,North,369.0
2,Northeast,1317.0
3,South,2699.0
4,Southeast,4557.0


In [17]:
# Translate "Região" to "Region" to ensure matching column names for the merge
df_region_pop.rename(columns={"Região": "Region"}, inplace=True)

# Merge the DataFrames on the "Region" column
merged_df = pd.merge(region_data_final, df_region_pop, on="Region")

# Calculate "% per 100 k habitants" and round to 2 decimal places
merged_df["% per 100 k habitants"] = ((merged_df["Total Procedures"] / merged_df["População_per_100k"]) * 100).round(2)

# Select the columns for the final DataFrame
final_df_all = merged_df[["Region", "Total Procedures", "% per 100 k habitants"]]

final_df_all

,Region,Total Procedures,% per 100 k habitants
0,Central-West,142.0,87.18
1,North,369.0,212.68
2,Northeast,1317.0,241.01
3,South,2699.0,901.67
4,Southeast,4557.0,537.08


In [18]:
final_df.to_csv('csv/tuberculose_by_region.csv')

### Table with full information

In [19]:
# Mapping of states to their corresponding regions
state_to_region = {
    'Acre': 'North', 'Amapá': 'North', 'Amazonas': 'North', 'Pará': 'North',
    'Rondônia': 'North', 'Roraima': 'North', 'Tocantins': 'North',
    'Alagoas': 'Northeast', 'Bahia': 'Northeast', 'Ceará': 'Northeast', 'Maranhão': 'Northeast',
    'Paraíba': 'Northeast', 'Pernambuco': 'Northeast', 'Piauí': 'Northeast',
    'Rio Grande do Norte': 'Northeast', 'Sergipe': 'Northeast',
    'Goiás': 'Central-West', 'Mato Grosso': 'Central-West', 'Mato Grosso do Sul': 'Central-West',
    'Distrito Federal': 'Central-West',
    'Espírito Santo': 'Southeast', 'Minas Gerais': 'Southeast', 'Rio de Janeiro': 'Southeast', 'São Paulo': 'Southeast',
    'Paraná': 'South', 'Rio Grande do Sul': 'South', 'Santa Catarina': 'South'
}

data_dropped = combined_df.drop(columns=['Code', 'Description'])

# Melting the adjusted dataset to organize it into a long format
data_adjusted_melted = data_dropped.melt(id_vars=['Month'], var_name='State', value_name='Procedures')
data_adjusted_melted = data_adjusted_melted[data_adjusted_melted['Procedures'] > 0]  # Keeping only rows with procedures

# Mapping states to their respective regions again
data_adjusted_melted['Region'] = data_adjusted_melted['State'].map(state_to_region)

# Selecting and renaming columns to fit the request
final_adjusted_data = data_adjusted_melted[['State', 'Region', 'Month', 'Procedures']]

# Display the first few rows of the newly adjusted table
final_adjusted_data.head()

,State,Region,Month,Procedures
376,Alagoas,Northeast,February,1.0
388,Alagoas,Northeast,February,1.0
468,Alagoas,Northeast,March,2.0
481,Alagoas,Northeast,March,2.0
502,Alagoas,Northeast,January,3.0


In [20]:
final_adjusted_data.to_excel('excel/tuberculose_state_region_month_freq.xlsx')

### Final adjustaments to match the map on BI

In [21]:
import unidecode

# Remover a acentuação da coluna 'State'
final_adjusted_data['State'] = final_adjusted_data['State'].apply(lambda x: unidecode.unidecode(x))

# Salvar o arquivo modificado
modified_file_path = 'excel/tuberculose_state_region_month_freq.xlsx'
final_adjusted_data.to_excel(modified_file_path, index=False)

modified_file_path

'excel/tuberculose_state_region_month_freq.xlsx'